In [2]:
import csv
import cv2 as cv
import mediapipe as mp

mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

hands = mp_hands.Hands(
    static_image_mode=False,
    max_num_hands=2,
    min_detection_confidence=0.5
)

cam = cv.VideoCapture(0)

# Open CSV file in write mode
with open('tempCollected.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    
    # Write header row
    header_row = ['Landmark' + str(i+1) + axis for i in range(21) for axis in ['X', 'Y', 'Z']]
    header_row.append('class')
    writer.writerow(header_row)

    while cam.isOpened():
        success, frame = cam.read()
        if not success:
            print("No frame to show")
            continue
        
        frame = cv.cvtColor(cv.flip(frame,1), cv.COLOR_BGR2RGB)
        hands_detected = hands.process(frame)
        frame=cv.cvtColor(frame, cv.COLOR_RGB2BGR)
        
        if hands_detected.multi_hand_landmarks:
            for hand_landmarks in hands_detected.multi_hand_landmarks:
                # Write landmarks to CSV
                landmark_row = []
                for landmark in hand_landmarks.landmark:
                    landmark_row.extend([landmark.x, landmark.y, landmark.z])
                
                # Fill empty cells if less than 21 landmarks detected
                while len(landmark_row) < 63:
                    landmark_row.extend([None, None, None])
                
                # Write 'class' value
                landmark_row.append('class')
                
                writer.writerow(landmark_row)

                # Draw landmarks on the frame
                mp_drawing.draw_landmarks(
                    frame,
                    hand_landmarks,
                    mp_hands.HAND_CONNECTIONS
                )
        
        cv.imshow("Captures", frame)
        if cv.waitKey(1) & 0xFF == ord('q'):
            break

    # Release resources
    cam.release()
    cv.destroyAllWindows()

print("CSV file 'tempCollected.csv' has been created successfully.")


CSV file 'tempCollected.csv' has been created successfully.
